# Day 1: Data Cleaning

# Importing libraries

In [137]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Our data is used to predict who survived and who didnt

In [138]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# removing all irrelevant columns. 

In [140]:
# for our simple ml model lets remove cabin, embarked, ticket, parch, sibsp, Name amd pID
# we use df.drop([column_name1, column_name2,....], axis=1) for column drop 
df = df.drop(['Cabin', 'Embarked', 'Ticket', 'Parch', 'SibSp', 'PassengerId', 'Name'], axis=1)
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   Fare      891 non-null    float64
dtypes: float64(2), int64(2), object(1)
memory usage: 34.9+ KB


# Removing NaN values and filling it with mode

In [142]:
df["Age"] = df["Age"].fillna(df["Age"].mode()[0])
df["Pclass"] = df["Pclass"].fillna(df["Pclass"].mode()[0])
df["Fare"] = df["Fare"].fillna(df["Fare"].mode()[0])

# We need to encode all string column and wrong categorical names to correct encodings of 0 and 1

In [143]:
#we need to convert sex to numbers, we use one hot encoding

OH = OneHotEncoder()
Sex = OH.fit_transform(df[["Sex"]])
Pclass = OH.fit_transform(df[["Pclass"]])
df["Female"], df["Male"] = Sex.toarray()[:,0], Sex.toarray()[:,1]
df["Pclass1"], df["Pclass2"], df["Pclass3"] = Pclass.toarray()[:,0], Pclass.toarray()[:,1], Pclass.toarray()[:,2]
df.head()

,Survived,Pclass,Sex,Age,Fare,Female,Male,Pclass1,Pclass2,Pclass3
0,0,3,male,22.0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1,1,female,38.0,71.2833,1.0,0.0,1.0,0.0,0.0
2,1,3,female,26.0,7.9250,1.0,0.0,0.0,0.0,1.0
3,1,1,female,35.0,53.1000,1.0,0.0,1.0,0.0,0.0
4,0,3,male,35.0,8.0500,0.0,1.0,0.0,0.0,1.0


In [144]:
# we will now drop the sex,pclass column
df = df.drop(["Sex", "Pclass"], axis=1)
df.head()

,Survived,Age,Fare,Female,Male,Pclass1,Pclass2,Pclass3
0,0,22.0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1,38.0,71.2833,1.0,0.0,1.0,0.0,0.0
2,1,26.0,7.9250,1.0,0.0,0.0,0.0,1.0
3,1,35.0,53.1000,1.0,0.0,1.0,0.0,0.0
4,0,35.0,8.0500,0.0,1.0,0.0,0.0,1.0


# Normalizing values using Normalizer

In [145]:
X = df.to_numpy()
Normalize = Normalizer().fit(X[:,1:3])
X[:,1:3] = Normalize.transform(X[:,1:3])

# Splitting data into Xtrain, Xtest, ytrain, ytest

In [146]:
#split into train and test
#first lets seperate x and y
y = X[:,0]
X = X[:,1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7,random_state=910)

# Fitting a simple logistic regression model

In [147]:
#simple logistic regression model


LR = LogisticRegression()
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
print(accuracy_score(y_pred, y_test))

0.746268656716418
